[View in Colaboratory](https://colab.research.google.com/github/jeffleo/Arduino-INA219/blob/master/Machine_Learning_Week_2_Post.ipynb)

![Machine Learning](https://www.comp.nus.edu.sg/~kanmy/courses/3244_1810/cs3244banner.png)
---
See [Credits](https://colab.research.google.com/drive/10T1tSBCwsc1lOp7XXV2ED3P0nAppr2Tq#scrollTo=-kGaO2wXmyVP) for acknowledgements and rights.
For NUS class credit, you'll need to download this notebook, complete all of the exercises in it (search for "Your Turn"), and submit the notebook online in IVLE by  **$\large\color{red}{\sf Sun, 26\:Aug\:2018, 23:59\: SGT}$**.  You must work in pairs or triples for all the in-class notebook assignments in class, unless otherwise specified by the lecturer(s).  

**You must acknowledge that your submitted notebook is your team's independent work, accomplished partially during our in-class session; see below at the end.**


**Learning Outcomes for this Week**

After finishing the in-class exercises and post-class videos, you should be able to:
* Understand Bayes' Rule and its application to Naïve Bayes;
* Understand distance metrics and its application to $k$ Nearest Neighbors;
* Implement both Naïve Bayes and $k$ Nearest Neighbors learning algorithms in pseudocode;
* Know the training and testing times for both of the above algorithms;
* Understand and apply basic linear algebra operations and know its terminology (rank, determinant, trace; vector, matrix, tensor; \*Hessian, \*Jacobian).


---
# In Class Session
# 1 Naïve Bayes

Today, we are going to work on implementing Naïve Bayes from scratch.  We'll be using the dataset that you are already familiar with from the pre-class video, the _Play Golf_ dataset.

In [0]:
import numpy as np
import pandas as pd

# Let's create the Play Golf dataset from scratch
pg_data = np.array([['Rainy','Hot','High','False','No'],
              ['Rainy','Hot','High','True','No'],
              ['Overcast','Hot','High','False','Yes'],
              ['Sunny','Mild','High','False','Yes'],
              ['Sunny','Cool','Normal','False','Yes'],
              ['Sunny','Cool','Normal','True','No'],
              ['Overcast','Cool','Normal','True','Yes'],
              ['Rainy','Mild','High','False','No'],
              ['Rainy','Cool','Normal','False','Yes'],
              ['Sunny','Mild','Normal','False','Yes'],
              ['Rainy','Mild','Normal','True','Yes'],
              ['Overcast','Mild','High','True','Yes'],
              ['Overcast','Hot','Normal','False','Yes'],
              ['Sunny','Mild','High','True','No']])
pg = pd.DataFrame(pg_data)
# add column headers to our data
pg.columns = ["outlook", "temperature", "humidity", "windy","play_golf"]
# Partition the features from the class to predict
pg_X = pg[pg.columns[pg.columns != "play_golf"]].copy()
pg_y = pg["play_golf"].copy()
# check that that worked by printing the first few rows
pg.head()

Recall that in Naïve Bayes, we use Bayes' Rule: $P(y|x)=\frac{P(x|y)\times P(y)}{P(x)}$, and that we make a naïve assumption that all of the features of $x$ are independent of each other.  We then change the right hand side (RHS) of the rule to $\frac{\sum_i^nP(x_i|y)\times P(y)}{P(x)}$.  

We need statistics of the prior probability $P(y)$ and all of the conditional probabilities for all $n$ features of $x$: $P(x_1|y)$ through $P(x_n|y)$.  We don't need the denominator $P(x)$, the probability of the data, because the probabilities will affect all outcomes equally.

Let's go ahead and get the prior probabilities of the class labels $P(y)\in\{Yes,No\}$.

In [0]:
# Let's calculate the priors.  They should be probabilities between 0 and 1. 
# First calculate the number of rows in the data table.
num_rows = len(pg)

# We'll use a hashtable to store the values of the priors.  Not efficient but simple.

# the dataframe makes it easy to check whether rows or cells meet a condition.
priors = { 'Yes': np.sum(pg['play_golf']=='Yes'),
           'No':  np.sum(pg['play_golf']=='No')}

# Done! Let's check...
print ("Prior probability of 'Play Golf == Yes' = %f" % priors['Yes'])

# Ooo, my bad.  These are counts, not probabilities yet.  
# Your Turn: Can you fix lines 8 and 9?

Next we are going to calculate our conditional probabilities on the individual features of $x$, i.e., $P(x_i|y)$.  
We'll start by counting how many times for the individual features of $x$ occur conditioned on the specific target $y$ value. 

In [0]:
# Let's calculate the conditional probabilities.  They should also be probabilities between 0 and 1.  
# We'll use a hashtable of two hashtables to store the values.
# The first level we'll store the hashes for the output conditions, 
# and in the second level we'll store counts for the input feature values such as such as 'outlook==Rainy'
likelihoods = {'Yes':{}, 'No':{}}

likelihoods['Yes']['outlook==Rainy']    = len(pg[(pg['outlook']=='Rainy') & (pg['play_golf']=='Yes')])    
likelihoods['Yes']['outlook==Sunny']    = len(pg[(pg['outlook']=='Sunny') & (pg['play_golf']=='Yes')])    
likelihoods['Yes']['outlook==Overcast'] = len(pg[(pg['outlook']=='Overcast') & (pg['play_golf']=='Yes')]) 
likelihoods['Yes']['temperature==Hot']  = len(pg[(pg['temperature']=='Hot') & (pg['play_golf']=='Yes')])  
likelihoods['Yes']['temperature==Mild'] = len(pg[(pg['temperature']=='Mild') & (pg['play_golf']=='Yes')]) 
likelihoods['Yes']['temperature==Cool'] = len(pg[(pg['temperature']=='Cool') & (pg['play_golf']=='Yes')]) 
likelihoods['Yes']['humidity==High']    = len(pg[(pg['humidity']=='High') & (pg['play_golf']=='Yes')])    
likelihoods['Yes']['humidity==Normal']  = len(pg[(pg['humidity']=='Normal') & (pg['play_golf']=='Yes')])  
likelihoods['Yes']['windy==True']       = len(pg[(pg['windy']=='True') & (pg['play_golf']=='Yes')])       
likelihoods['Yes']['windy==False']      = len(pg[(pg['windy']=='False') & (pg['play_golf']=='Yes')])      

likelihoods['No']['outlook==Rainy']    = len(pg[(pg['outlook']=='Rainy') & (pg['play_golf']=='No')])    
likelihoods['No']['outlook==Sunny']    = len(pg[(pg['outlook']=='Sunny') & (pg['play_golf']=='No')])    
likelihoods['No']['outlook==Overcast'] = len(pg[(pg['outlook']=='Overcast') & (pg['play_golf']=='No')]) 
likelihoods['No']['temperature==Hot']  = len(pg[(pg['temperature']=='Hot') & (pg['play_golf']=='No')])  
likelihoods['No']['temperature==Mild'] = len(pg[(pg['temperature']=='Mild') & (pg['play_golf']=='No')]) 
likelihoods['No']['temperature==Cool'] = len(pg[(pg['temperature']=='Cool') & (pg['play_golf']=='No')]) 
likelihoods['No']['humidity==High']    = len(pg[(pg['humidity']=='High') & (pg['play_golf']=='No')])    
likelihoods['No']['humidity==Normal']  = len(pg[(pg['humidity']=='Normal') & (pg['play_golf']=='No')])  
likelihoods['No']['windy==True']       = len(pg[(pg['windy']=='True') & (pg['play_golf']=='No')])       
likelihoods['No']['windy==False']      = len(pg[(pg['windy']=='False') & (pg['play_golf']=='No')])      

# Check that our calculation is right
print("Likelihood of it being sunny, given we are playing golf = %f" % likelihoods['Yes']['outlook==Sunny'])

# That's not right.  Ah... We forgot to normalise by the probability of the target class.  
# Your Turn: Can you fix the above sets of lines 7-16 and 18-27 by dividing through by the appropriate amount?

Great! We have everything we need.  It's sunny today outside.  What's the posterior probability that we'll go out and play golf? i.e., $P(Yes|outlook=Sunny)$.  Note the difference between the previous likelihood and what we want to calculate now.

In [0]:
print("Posterior* probability we will play golf when it's sunny = %f" % (likelihoods['Yes']['outlook==Sunny'] * priors['Yes']))
print("Posterior* probability we won't play golf when it's sunny = %f" % (likelihoods['No']['outlook==Sunny'] * priors['No']))

The above looks strange doesn't it?  The probabilities don't sum to unity (1).  If they were really posterior probabilities, they would have to add up.  We put "Posterior$\color{red}{*}$" instead of just "Posterior" to alert you to this.  This is because we didn't actually divide through by the data probabilities, the marginals, as in the slides, as they will affect both outcomes equally.  

So it looks like we'll play golf after all.  

Wait... Now that we've gone outside, I notice that the air is still and that it's humid and the temperature is hot.  That's Singapore for you.  What's my "posterior" probability now? 

In [0]:
# Your Turn: fill in the calculations
posterior_yes_SHHF = 1 # Fix this line
posterior_no_SHHF = 1  # Fix this one too

print("Posterior* probability we will play golf when it's sunny, not windy, humid and hot = %f" % posterior_yes_SHHF)
print("Posterior* probability we won't play golf when it's sunny, not windy, humid and hot = %f" % posterior_no_SHHF)

_Aiya!_  Guess I'll stay inside and do some machine learning instead. ^\_^ 

Maybe we can fix the above to calculate actual probabilities for these values.  We'll just have to normalise them against each other.  That's the last part for today.

In [0]:
## Your turn: Calculate the actual probability for our case of playing golf when it's sunny, still, humid and hot.
# Store it in the variable "Q05"

Q05 = 1 # Fix this line

print("Actual posterior probability we will play golf when it's sunny, not windy, humid and hot = %f" % Q05)

(Optional)  We said that Naïve Bayes can only handle enumerated features and labels: see the summary slide in the Week 2 Pre videos.  Yet, the sklearn Naïve Bayes that we used in the pre class exercise handled all of the continuous features.  There's some contradiction here.  Let's pause for a moment and think what could be the difference.  Feel free to look it up on the Web (but then please follow best practice and credit where you found the information in the _References_ part of your Declaration of Individual Work )

Your Turn (Optional): Why is there a discrepancy between the lecture and the pre-class exercise?  What does **sklearn**'s Naïve Bayes do that we didn't?  

---
# Post-Class Work


You will have to watch the post-class videos on the lecture topics introduced today, then attempt the following exercises.  

## 2 Prerequisites

This week is also a bit special.  This is because this week we want you to devote some time to reviewing the prerequisites for this course.  Everyone has slightly different foundations coming into this course, so it's important to attempt to put us all in the same square with respect to our own prior knowledge. 

**Linear Algebra Review**.
* For those who have a good grasp of Linear Algebra already, you may just want to refresh yourself using [Chapter 2 Linear Algebra](http://www.deeplearningbook.org/contents/linear_algebra.html) of the Goodfellow et al. _[Deep Learning](http://www.deeplearningbook.org/)_ book (The book itself is very good, we'd recommend it for any of you wanting to learn more about deep learning, plus it's freely accessible online).
* If you're a little shakier with your LA foundations, you can work your way back up.  You should be familar with Eigendecomposition from your previous linear algebra course.  Have a look at 3Blue1Brown's [Essence of Linear Algebra](http://3b1b.co/eola) playlist.  This is a beautiful series of videos that have been [animated in Python](https://github.com/3b1b/manim) (go Python!) by its creator.  Watch what you need to understand the penultimate [Eigenvectors and eigenvalues](https://www.youtube.com/watch?v=PFDu9oVAE-g&list=PLZHQObOWTQDPD3MizzM2xVFitgF8hE_ab&index=14).  As stated by the 3Blue1Brown, Eigen-"things" are not too difficult to understand once you have a solid foundation on linear transformations, determinants, linear systems and change of basis.
* Review the textbook from _[Immersive Math](http://immersivemath.com/ila/)_.  This is a recent, online textbook for linear algebra that is close to finished.  Many of the linear algebra concepts will be much better explained by this textbook through its 3D interactive animations.

**Probability and Statistics Review**
These two terms are inverse processes between data and a model.  _Probability_ is the process of analyzing a model to discover what data it will generate; _Statistics_ uses data to generate a model.  The second definition is close to what we mean by ML, so it's no surprise that statistics is a primary contributor to the ML world; in fact, in much more rigorous detail than we will go over in this course.   
* It's good to have a good grasp of the [normal distribution](http://www.statisticshowto.com/probability-and-statistics/normal-distributions/).  I'd suggest being familiar with its properties.  These will be handy when we talk about random noise affecting learned models.
* For this lecture where we discuss Bayes' Rule in part, understanding different parts that make up the posterior, and how it can be calculated from the data likelihoods, marginals and priors.

In [0]:
#@title Your Turn: I actually reviewed the resources above to my satisfaction.
Q06 = 'Yes I did' #@param ["Yes I did", "No I didn't", "I wanted to ...", "I did a cursory review"]

### Quiz Time (hidden until you open it) 
Now that you think you've done an adequate review; let's have a mock test (You can be honest when the stakes are zero, these questions have zero weight).  When you're ready, open up the below section and take the mini-quiz.  Resist the temptation to look up answers on the Web.  But don't forget to take this mini-quiz before turning in your post-class notebook, ok?

In [0]:
#@title Your Turn: Is matrix multiplication associative?  Is it commutative?   
Q07 = "Yes to both" #@param ["Yes to both", "No to both", "Yes to associative and no to commutative", "Yes to commutative and no to associative"]

In [0]:
#@title Your Turn: Explain what the L2 norm is in layman terms.   
Q08 = 'Replace with your answer' #@param {type:"string"}

**The _Monty Hall Problem_**.  

![3 doors](https://upload.wikimedia.org/wikipedia/commons/3/3f/Monty_open_door.svg =600x300)

(photo credits: in the public domain, authored by user Cepheus @ _Wikipedia_)

Suppose you're on a game show, and you're given the choice of three doors: Behind one door is a car; behind the others, goats. You pick a door, say No. 1, and the host, who knows what's behind the doors, opens another door, say No. 3, which has a goat. He then says to you, "Do you want to pick door No. 2?"  

In [0]:
#@title Your Turn: Is it to your advantage to switch your choice? 
Q09 = "Yes" #@param ["Yes","No","It depends"]

Your Turn (Optional): From above Q09, why or why not? Justify, as you would to a peer in the class.  You may find it useful to use LaTeX math notation, which is easier learned from examples (you can see the source for the equations we used in our notebooks by double clicking cells in your copy of the notebook).

(Optional) Replace with your answer...

Some of you may already be familiar with this problem from previous classes before university.  [Monty Hall](https://en.wikipedia.org/wiki/Monty_Hall) was a famous TV personality, hosting a game show _Let's make a deal_ in the 1960s to the 1980s.  He offered versions of this problem to contestants.  This problem flummoxed many members of the public, including mathematics Ph.D. holders and (famously, [Paul Ërdos](https://en.wikipedia.org/wiki/Paul_Erd%C5%91s)), who wrote in to [Marilyn von Savant](https://en.wikipedia.org/wiki/Marilyn_vos_Savant), the columnist who offered the correct solution to the problem, to tell her she was wrong.


Post quiz, if you got stuck on any of these problems, you can look it up on the Web (don't forget to give credit in your _Declaration_), and discuss it on the discussion forum so that everyone can benefit.

## 3 $k$ Nearest Neighbors




As we discussed in the video $k$ NN doesn't have to do anything to set up the model $h_{\theta}$ for training.  The parameters ${\theta}$  in $k$ NN is actual the _hyperparameter_ choices made for the distance metric and the number of clusters $k$.  

Let's examine the Iris dataset which has samples of iris flowers attributed to 3 different species of iris.  For simplifiy, we'll study only the first two features, and examine how the decision boundary changes with different values of $k$.

Feel free to look at other features and change the hyperparameter values - the code is provided to you for studying and experimentation. 

In [0]:
# This code is a modification of the sample code for kNN in sklearn
# Modified by Min, Aug 2018.
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn import neighbors, datasets

iris = datasets.load_iris()

# we only take the first two features. 
X = iris.data[:, :2]
y = iris.target

# Create color maps
cmap_light = ListedColormap(['#FFCCCC', '#CCFFCC', '#CCCCFF'])
cmap_bold = ListedColormap(['#FF0000', '#00FF00', '#0000FF'])

step_size = .02  # step size in the mesh

plt.figure(figsize=(10,10)) # make a bigger plot for our subplots
for i in range(1,10):
  ## map a subplot i to a certain selection of k
  # Your turn (optional): modify and experiment
  n_neighbors = 4*i - 3 
  plt.subplot(3, 3, i) 
  # we create an instance of Neighbours Classifier and fit the data.
  clf = neighbors.KNeighborsClassifier(n_neighbors)
  clf.fit(X, y)

  # Plot the decision boundary. For that, we will assign a color to each
  # point in the mesh [x_min, x_max]x[y_min, y_max].
  x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
  y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
  xx, yy = np.meshgrid(np.arange(x_min, x_max, step_size),
                       np.arange(y_min, y_max, step_size))
  Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])

  # Put the result into a color plot
  Z = Z.reshape(xx.shape)
  plt.pcolormesh(xx, yy, Z, cmap=cmap_light)

  # Plot also the training points
  plt.scatter(X[:, 0], X[:, 1], c=y, cmap=cmap_bold,
              edgecolor='k', s=20)
  plt.xlim(xx.min(), xx.max())
  plt.ylim(yy.min(), yy.max())
  plt.title("%i-NN" % n_neighbors)

plt.tight_layout()
plt.show()

## 4 Scaling and Normalizing Data


Certain machine learning algorithms are sensitive to distance changes, as we have discussed in the post-class video lecture.  Let's see the effect that normalization and scaling has on the wine quality dataset.  Let's first get it set up.

In [0]:
import numpy as np 
import pandas as pd 
from sklearn.model_selection import train_test_split 

# Let's read the data in as a data frame, equivalent to our (X,Y) data matrix. 
# We'll use another variable even though it's the same data as in our W2 Pre notebook.
wq = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-white.csv',sep=';') # Separate on semicolons

wq['good wine'] = np.where(wq['quality']>=7, "Good", "Not Good")
wq.drop('quality', axis=1, inplace=True) 
wq_X = wq[wq.columns[wq.columns != 'good wine']].copy() 
wq_y = wq['good wine'].copy()
X_train, X_test, y_train, y_test = train_test_split(df_X, df_y, test_size=0.2, random_state=0)

Scaling usually is done using min–max scaling, where we take the minimum and maximum values and assign them to 0 and 1, respectively.  Every other value gets mapped in between by a linear transformation.

Often, instead of scaling, we will do normalization, which takes a feature (column) $x_i$ and rescales it so that it's values will have the properties of a standard normal distribution with $\mu=0$ and $\sigma = 1$.  Both of these operations are available in sklearn.  Let's see the effect of normalization has on the wine quality dataset:

In [0]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train) # rescale for \mu = 0 and \sigma = 1, and remember the parameters for the scaling
X_test_scaled = scaler.transform(X_test) # Apply the transform with the **same parameters from training** to the test data.  

In [0]:
fa_mean =        X_train['fixed acidity'].mean()
fa_mean_scaled = X_train_scaled[:,0].mean() # fixed acidity is column 0, when we transformed the data, the output doesn't have column information any more
fa_std =        X_train['fixed acidity'].std()
fa_std_scaled = X_train_scaled[:,0].std() 

va_mean =        X_train['volatile acidity'].mean()
va_mean_scaled = X_train_scaled[:,1].mean() # volatile acidity is column 1, when we transformed the data, the output doesn't have column information any more
va_std =        X_train['volatile acidity'].std()
va_std_scaled = X_train_scaled[:,1].std() 

# check
print ("Fixed Acidity mean and standard deviation: %f (%f)" % (fa_mean,fa_std))
print ("Fixed Acidity (normalized) mean and standard deviation: %f (%f)" % (fa_mean_scaled,fa_std_scaled))

In [0]:
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
%matplotlib inline

# plot both the normalized and unnormalized data in the same plot
plt.figure(figsize=(8,6))
plt.scatter(X_train['fixed acidity'], X_train['residual sugar'],
            color='green', label='Unnormalized', alpha=0.5)
plt.scatter(X_train_scaled[:,0], X_train_scaled[:,1], color='red',
            label='Normalized [$\mu = 0, \sigma = 1$]', alpha=0.3)
plt.title('Fixed versus Volatile Acidity on the Wine Quality Dataset')
plt.xlabel('Fixed Acidity')
plt.ylabel('Volatile Acidity')
plt.legend(loc='upper left')
plt.grid()

plt.tight_layout()


In [0]:
# Get the machine learning algorithm
from sklearn import neighbors

knn = neighbors.KNeighborsClassifier(n_neighbors = 1)
knn_model_scaled = knn.fit(X_train_scaled, y_train)
print('1-NN accuracy (Scaled) for test set: %f' % knn_model_scaled.score(X_test_scaled, y_test))
knn_model = knn.fit(X_train, y_train)
print('1-NN accuracy (Non-scaled) for test set: %f' % knn_model.score(X_test, y_test))

In [0]:
#@title Your Turn: Why do you think scaling helps in this case to improve nearest neighbors?
Q10 = 'Replace with your answer' #@param {type:"string"} 

Your Turn (Optional): From above Q10, why or why not? Justify, as you would to a peer in the class.  Do you think it always helps?  When would you use it?

Replace with your answer...

## 5 Curse of Dimensionality

(Optional) Finally we include code for you to run and modify to check the _Curse of Dimensionality_.  Recall that the Curse of Dimensionality suggests that the larger the dimensional space you have in your dataset, the more difficult certain aspects of learning can become.

In [0]:
import numpy as np

# Try some on your own
for dim in [1,2,3,4,8,10,100,200]:
    ## choices of |X|
    samples = 200
    #samples = pow(2,dim)
    X = np.random.uniform(size=(samples,dim))

    ## choices of x
    x = np.random.uniform(size=(dim))
    # x = np.zeros(dim)
    print("%d dimensions:" % dim)
    print("#   Generated %d points" % samples)
    # print("#   Test point: ", x)
    dist_array = []
    for j in range(len(X)):
        dist = 0.0
        for i in range(dim):
    #        print ("%f => %f "% (x[i]-X[j][i],pow(x[i]-X[j][i],2)))
            dist += pow(x[i]-X[j][i],2)
    #    print ("Sum distance %f => %f" % (dist,pow(dist,0.5)))
        dist = pow(dist,0.5)
        dist_array.append(dist)
    #    print ("%d th point %s has distance %f" % (j,X[j],dist))
    distances = np.array(dist_array)
    print ("#   Average distance (STD) of test point to %d samples: %f (%f)" % (samples,distances.mean(),distances.std()))

---
This comes to the end of this class' notebook.   **Each member of your team must submit the notebook to IVLE workbin by the appropriate deadline of **$\large\color{red}{\sf Sun, 26\:Aug\:2018, 23:59\: SGT}$**.  Working on your own defeats the purpose of our flipped classroom exercises and is not allowed; you will be asked to work in a team if you come late.  This is necessary to let our automated grading programs mark your assignment.
Don't forget to do your pre-class video watching and pre-class work in the subsequent week's notebook. 

# Declaration of Independent Work
By entering my ID below, I certify that I completed my notebook independently of all others (except where sanctioned during in-class sessions), obeying the class policy outlined in the introductory lecture.  In particular, I am allowed to discuss the problems and solutions in this notebook, but have waited at least 30 minutes by doing other activities unrelated to class before attempting to complete or modify my answers as per the Pokémon Go rule.  You must sign **exactly one of the two below statements** with your Student ID.  Leave the other as-is.

If you need to sign the second declaration of not following policy, please also complete the other two text boxes marked with "Your Turn".

**Your Turn (References)**: I give credit where credit is due.  I acknowledge that I used the following websites or contacts to complete this assignment:
* _Sample_. [Website 1](http://example.com), for following mathematical proofs.
* _Sample_. My friend, John Doe, on the programming syntax.


In [0]:
#@title I followed class policy

alt_sign1 = "Your Student ID" #@param {type:"string"}

In [0]:
#@title I didn't follow class policy
alt_sign2 = "Your Student ID" #@param {type:"string"}

**Your Turn**: I didn't follow class policy because of ...

**Your Turn**: I suggest I be graded as ...

---
# Credits
Authored by [Min-Yen Kan](http://www.comp.nus.edu.sg/~kanmy) and Chris Boesch (2018), affiliated with [WING](http://wing.comp.nus.edu.sg), [NUS School of Computing](http://www.comp.nus.edu.sg) and [ALSET](http://www.nus.edu.sg/alset).
Licensed as: [Creative Commons Attribution 4.0 International](https://creativecommons.org/licenses/by/4.0/ ) (CC BY 4.0).  Based partially on notebooks from [SarahG](https://www.kaggle.com/sgus1318/wine-quality-exploration-and-analysis/notebook) via Kaggle, [Nishank Kumar Sharma](https://medium.com/click-bait/wine-quality-prediction-using-machine-learning-59c88a826789) via Medium, Iris plot modified from tutorial on sklearn. 
Please retain and add to this credits cell if using this material as a whole or in part.   Credits for photos given in their captions.

## Grading Harness (not to be edited)

In [0]:
Q00 = "revision=1"
Q03 = priors['Yes']
Q04 = likelihoods['Yes']['outlook==Sunny']

# General
if alt_sign1 == "Your Student ID" or alt_sign1=="":
  if alt_sign2 == "Your Student ID" or alt_sign2=="": # both unsigned
    matric = "Unknown"
    status = "unsigned"
  else: # Signed 2
    matric = alt_sign2
    status = "did not follow"
else:
  if alt_sign2 == "Your Student ID" or alt_sign2=="":
    matric = alt_sign1
    status = "followed" 
  else:
    matric = alt_sign1 
    status = "signed both"

csvdata = [matric,status]
csvdata.extend(value for name, value in sorted(locals().items(), key=lambda item: item[0]) if name.startswith('Q'))
def replComma(Q): return Q.replace(',','') if isinstance(Q,str)  else Q 
csvdata = [replComma(Q) for Q in csvdata]
print (csvdata)

import json

with open('results.json', 'w') as outfile:
    json.dump(csvdata, outfile)
    
import re
from IPython.display import Image
from IPython.core.display import HTML 

def calculateNUSMatricNumber(id):
  matches = re.match(r'^A\d{7}.|U\d{6,7}.',id)
  if (matches):
    originalMatch = id;
    match = matches[0][:-1]
    
    #Discard 3rd digit from U-prefix NUSNET ID
    if (match[0] == 'U' and len(match) == 8):
      match = mathc[:3] + match[4]
    weigths = { 'U':[0,1,3,1,2,7],'A':[1,1,1,1,1,1]}[match[0]]
    sum = 0;
    digits = match[-6:];
    for i in range(6):
      sum += weigths[i] * int(digits[i])     
    return originalMatch == (match + 'YXWURNMLJHEAB'[sum %13])

def checkNotebook():
    if not 'csvdata' in globals():
      print("Seems like you haven't run the Grading Harness. Just got to 'Runtime -> Run all' and run the entire notebook.")
      return display(HTML('<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/c/cc/Smiley_green_alien_sherlock.svg/200px-Smiley_green_alien_sherlock.svg.png" alt="Sad"><br> \
                          <i>Graphics in the public domain from the <a href="https://commons.wikimedia.org/wiki/User:LadyofHats">LadyofHats</a> \
                          via <a href="https://commons.wikimedia.org/wiki/File:Smiley_green_alien_sherlock.svg">Wikimedia Commons</a>.') )
    elif csvdata[1] == "signed both":
      print("Seems like you have signed both fields. That's a little confusing, please decide on one and leave the other empty.")
      return display(HTML('<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/9/99/Smiley_green_alien_dead_ninja.svg/200px-Smiley_green_alien_dead_ninja.svg.png" alt="Confused"><br> \
                           <i>Graphics in the public domain from the <a href="https://commons.wikimedia.org/wiki/User:LadyofHats">LadyofHats</a> \
                           via <a href="https://commons.wikimedia.org/wiki/File:Smiley_green_alien_dead_ninja.svg">Wikimedia Commons</a>.') )
    elif(calculateNUSMatricNumber(csvdata[0])):
      print("Great. Everything seems to be in place. You can download the notebook and submit it to IVLE.")
      return display(HTML('<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/7/79/Face-smile.svg/200px-Face-smile.svg.png" alt="Good job"><br>\
                          <i>Graphics in the public domain from the <a href="http://tango.freedesktop.org/The_People">Tango Project</a> \
                          via <a href="https://commons.wikimedia.org/wiki/File:Face-smile.svg">Wikimedia Commons</a>.') )
    elif(csvdata[0] == 'Unknown'):
      print("Seems like you forgot to enter your Student ID. Just enter it under 'Declaration of Independent Work' and try again (Runtime -> Run all).")
      return display(HTML('<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/9/9f/Smiley_green_alien_big_eyes.svg/200px-Smiley_green_alien_big_eyes.svg.png" alt="Sad"><br> \
                          <i>Graphics in the public domain from the <a href="https://commons.wikimedia.org/wiki/User:LadyofHats">LadyofHats</a> \
                          via <a href="https://commons.wikimedia.org/wiki/File:Smiley_green_alien_big_eyes.svg">Wikimedia Commons</a>.') )
    else:
      print("Whatever you entered under Declaration of Independent Work it's not a valid Student ID (A???????E). Please try again.")
      return display(HTML('<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/b/b3/Smiley_green_alien_worried.svg/200px-Smiley_green_alien_worried.svg.png" alt="Sad"><br> \
                          <i>Graphics in the public domain from the <a href="https://commons.wikimedia.org/wiki/User:LadyofHats">LadyofHats</a> \
                          via <a href="https://commons.wikimedia.org/wiki/File:Smiley_green_alien_worried.svg">Wikimedia Commons</a>.') )

## Final Check

In [0]:
checkNotebook()